<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/0203_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()


def get_news(url):
    headers = {"user-agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article = html.select_one("#articleBodyContents").text
    return article

url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108"

news_article = get_news(url)

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
from kss import split_sentences

def get_sentences(text):
    # return sent_tokenize(text)
    return split_sentences(text)

def get_words(text, isNoun = False):
    if isNoun:
        return [token[0] for token in mecab.pos(text) if token[1][0] == 'N' and len(token[0]) > 0]
    else:
        return [token[0] for token in mecab.pos(text)]

In [ ]:
def get_keywords(word_list, min_ratio= 0.001, max_ratio=0.5):
    keywords = set()
    
    count_dict = {}
    
    for word in word_list:
        if word in count_dict.keys():
            count_dict[word] = count_dict[word] + 1
        else:
            count_dict[word] = 1
            

    for word, cnt in count_dict.items():
        word_percentage = cnt / len(word_list)
        
        if word_percentage >= min_ratio and word_percentage <= max_ratio:
            keywords.add(word)
    
    return keywords
    


In [ ]:
get_keywords(['바나나', '사과', '바나나', '바나나', '포도'])

{'사과', '포도'}

In [ ]:
a = 5
b = 4
a/b

1.25

In [ ]:
def get_sentence_weight (token_list, keywords):
    window_start = 0; window_end = -1
    
    for i in range(len(token_list)):
        if token_list[i] in keywords:
            window_start = i
            break
    
    for i in range(len(token_list) - 1, -1, -1):
        if token_list[i] in keywords:
            window_end = i
            break
            
    if window_start > window_end:
        return 0
    
    window_size = window_end - window_start + 1
    
    #keyword 갯수
    keyword_cnt = 0
    for w in token_list:
        if w in keywords:
            keyword_cnt += 1
    
    return keyword_cnt * keyword_cnt / window_size

In [ ]:
get_sentence_weight(['바나나', '사과', '바나나', '바나나', '포도'], {'사과', '포도'})

1.0

In [ ]:
def summarize(context, no_sentences=5):
    word_list = get_words(context, isNoun=True)
    keywords = get_keywords(word_list)
    
    sentence_list = get_sentences(context)
    
    sentence_weight = []
    for sentence in sentence_list:
        token_list = get_words(sentence)
        sentence_weight.append((get_sentence_weight(token_list, keywords), sentence))
        
    
    sentence_weight.sort(reverse=True)
    
    return sentence_weight[:no_sentences]
    

In [ ]:
summ_sents = summarize(news_article, 3)
for s in summ_sents:
    print(s)

(26.07758620689655, '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다.')
(21.333333333333332, '과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.')
(18.753246753246753, '유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.')


In [ ]:
!pip install kss

  Created wheel for kss: filename=kss-3.3.1.1-py3-none-any.whl size=42449241 sha256=84436bd1f920cbcd5fc2614a9714ac9384cb94189798fa27f4fde7d71cdca8b1
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\6e\9d\1d\52871154eff5273abb86b96f4f984c1cd67c5bde64239b060a
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=db7c4cf2bf03d5fb479e017026d7a6f8e11e1854e800a8a4ae929ea796266d01
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\03\8b\d7\ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built kss emoji


In [ ]:
from kss import split_sentences
sentence_list = split_sentences(news_article)
print(len(sentence_list))
print(sentence_list)


[Korean Sentence Splitter]: Initializing Pynori...


23
['과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다.', '새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.', '22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.', '유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.', '◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다.', '문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.', '이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다.', '과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.', '대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다.', '이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는

In [ ]:
text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나"

In [ ]:
def sentence_similarity(sent1, sent2):
    # node_pos = ["NNG", "NNP"]
    
    sent1_list = [token[0] for token in mecab.pos(sent1) if token[1][0] in ['N', 'V']]
    sent2_list = [token[0] for token in mecab.pos(sent2) if token[1][0] in ['N', 'V']]
    
    union = set(sent1_list).union(set(sent2_list))
    intersection = set(sent1_list).intersection(set(sent2_list))
    
    return len(intersection)/len(union)
    

sentence_similarity("나는 치킨을 좋아해", "나는 치킨을 싫어해")

0.5

In [ ]:
import numpy as np


def buildMatrix(sentences):
    sentences_len = len(sentences)
    
    score = np.ones(sentences_len, dtype=np.float32)
    
    weighted_edge = np.zeros((sentences_len, sentences_len), dtype=np.float32)
    
    for i in range(sentences_len):
        for j in range(sentences_len):
            if i == j:
                continue
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
    
    for i in range(sentences_len):
        score[i] = weighted_edge[i].sum()
        weighted_edge[i] /= score[i]
    
    return weighted_edge
            
                             
buildMatrix(sent_tokenize(text))     

array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
       [0.45454544, 0.        , 0.        , 0.54545456],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32)

In [ ]:
def scoring(weighted_edge, score, eps=0.0001, d=0.85, max_iter = 50):
    for iter in range(max_iter):
        new_score = (1 - d) + d * weighted_edge.T.dot(score)
        
        for diff in abs(new_score - score):
            if diff <= eps:
                return new_score
            
        score = new_score
        
    return new_score

In [ ]:

def summize(text, no_sentences=5):
    sentence_list = sent_tokenize(text)
    weighted_edge = buildMatrix(sentence_list)
    score_init = np.ones(weighted_edge.shape[0], dtype=np.float32)
    score = scoring(weighted_edge, score_init)
    return sorted(enumerate(score), key=lambda x : x[1], reverse=True)[:no_sentences]

In [ ]:
summize(text, 3)

[(0, 1.4098104), (3, 1.2793452), (1, 1.0245311)]

In [ ]:
get_sentences(text)

[Korean Sentence Splitter]: Initializing Pynori...


['딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나']